In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import torchvision
import torch.optim as optim
import argparse
import matplotlib
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import pandas as pd
import numpy as np
matplotlib.style.use('ggplot')
from sklearn.model_selection import train_test_split
import lightning.pytorch as pl
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.preprocessing import MinMaxScaler
import math

torch.manual_seed(33)

c:\Users\mdzak\anaconda3\envs\fyp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Import Data

In [2]:
gene_exp = pd.read_csv('new_clean_data/consistent_gene_exp.csv')
gene_exp.drop(columns='Unnamed: 0', inplace=True)
gene_exp

,CELL_LINE_NAME,ENSG00000000003.10,ENSG00000000005.5,ENSG00000000419.8,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000938.8,ENSG00000000971.11,ENSG00000001036.9,ENSG00000001084.6,...,Dactinomycin,Daporinad,Dasatinib,Rapamycin,Romidepsin,SN-38,Temsirolimus,Trametinib,Vinblastine,Vinorelbine
0,22rv1,5.28,0.0,73.38,9.76,24.51,0.01,0.08,54.86,118.50,...,-4.840963,-3.112784,4.203067,-4.382138,-4.917572,-4.972312,-0.974396,1.325603,-4.384381,-3.401996
1,2313287,7.01,0.0,108.99,16.76,13.32,0.00,0.23,170.91,93.00,...,-4.584971,-3.336795,1.797167,-3.486065,-6.017003,-4.132899,3.279539,-0.040150,-4.849422,-5.303091
2,42mgba,23.09,0.0,99.28,2.73,9.27,0.02,0.45,53.29,8.36,...,-4.240733,-3.336795,-2.233603,-3.671158,-6.415517,-4.737156,-0.074308,1.621553,-4.784344,-3.927690
3,5637,57.94,0.0,98.95,3.11,31.61,0.14,1.98,71.97,9.76,...,-4.321517,-3.336795,-1.829021,-0.116397,-5.559258,-4.537337,1.818908,-0.275098,-3.494500,-5.252216
4,639v,32.02,0.0,125.99,2.69,10.45,0.00,12.89,82.50,24.13,...,-4.725271,-3.336795,1.391965,-1.822279,-4.955669,-6.639479,1.248546,1.437469,-3.986408,-3.138535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,wsudlcl2,0.74,0.0,41.71,5.60,20.21,30.57,0.46,11.78,8.32,...,-5.120747,-6.196767,-1.463565,-5.903482,-5.240336,-4.959568,-5.974346,0.641326,-7.103903,-6.196508
539,yapc,50.29,0.0,103.70,7.76,12.84,0.17,13.58,71.13,29.67,...,-1.932980,-5.076682,1.407784,1.214362,-3.897758,5.073883,1.195720,3.598733,-0.095525,2.307072
540,yh13,28.92,0.0,64.09,4.94,13.35,0.22,139.44,118.19,13.53,...,-3.595530,-3.469021,-0.787238,-2.743620,-5.957548,-3.526360,-1.963500,1.153223,-5.412392,-4.207989
541,ykg1,61.08,0.0,109.13,5.91,17.40,0.13,53.25,92.96,23.09,...,-5.262451,3.115970,0.597641,-0.763934,-5.240336,-3.225249,2.191202,0.407173,-3.494500,-2.601804


In [3]:
cell_line_cols = ['CELL_LINE_NAME']
drug_resp_cols = ['Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']
gene_exp_cols = gene_exp.drop(columns=['CELL_LINE_NAME','Dactinomycin', 'Daporinad', 'Dasatinib',	'Rapamycin', 'Romidepsin', 'SN-38',	'Temsirolimus',	'Trametinib', 'Vinblastine', 'Vinorelbine']).columns

In [4]:
# drug response labels to tensor
drug_resp_labels = np.stack([gene_exp[col].values for col in drug_resp_cols], 1)
drug_resp_labels = torch.tensor(drug_resp_labels, dtype=torch.float)

In [5]:
# gene_exp values to tensor and scale from 0 to 1
gene_exp_values = np.stack([gene_exp[col].values for col in gene_exp_cols], 1)

In [6]:
scaler = MinMaxScaler()
gene_exp_values_t = gene_exp_values.T
scaled_gene_exp_values_t = scaler.fit_transform(gene_exp_values_t)
scaled_gene_exp_values = scaled_gene_exp_values_t.T
scaled_gene_exp_values = torch.tensor(scaled_gene_exp_values, dtype=torch.float)

### Define Dataloaders

In [7]:
batch_size = 32

full_dataset = torch.utils.data.TensorDataset(scaled_gene_exp_values, drug_resp_labels)
full_loader = DataLoader(full_dataset, shuffle=False, batch_size=batch_size)

### VAE model with Regressor

In [8]:
class VAE_with_regressor(nn.Module):
    def __init__(self, input_size, level_2, level_3, latent_dim):
        super(VAE_with_regressor, self).__init__()
        cat_features = 10
        # Encoder layers
        self.enc_fc1 = nn.Sequential(
                        nn.Linear(input_size, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.enc_fc2 = nn.Sequential(
                        nn.Linear(level_2, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())

        self.enc_fc3_mean = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        self.enc_fc3_log_var = nn.Sequential(
                    nn.Linear(level_3, latent_dim),
                    nn.BatchNorm1d(latent_dim))
        
        
        # Decoder layers
        self.dec_fc3 = nn.Sequential(
                        nn.Linear(latent_dim, level_3),
                        nn.BatchNorm1d(level_3),
                        nn.ReLU())
        
        self.dec_fc2 = nn.Sequential(
                        nn.Linear(level_3, level_2),
                        nn.BatchNorm1d(level_2),
                        nn.ReLU())
        
        self.dec_fc1 = nn.Sequential(
                    nn.Linear(level_2, input_size),
                    nn.BatchNorm1d(input_size),
                    nn.Sigmoid())
        
        # Regression fc layers
        self.r_fc1 = nn.Sequential(
                    nn.Linear(latent_dim, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU())
        self.r_fc2 = nn.Sequential(
                    nn.Linear(64, 32),
                    nn.BatchNorm1d(32),
                    nn.ReLU())
        
        self.r_fc3 = nn.Sequential(
                    nn.Linear(32, 10),
                    nn.BatchNorm1d(10))


    def encode(self, x):
        l2_layer = self.enc_fc1(x)
        l3_layer = self.enc_fc2(l2_layer)
        
        mu = self.enc_fc3_mean(l3_layer)
        logvar = self.enc_fc3_log_var(l3_layer)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z):
        l3_layer = self.dec_fc3(z)
        l2_layer = self.dec_fc2(l3_layer)
        x_hat = self.dec_fc1(l2_layer)
        return x_hat

    def regressor(self, mean):
        level_1_layer = self.r_fc1(mean)
        level_2_layer = self.r_fc2(level_1_layer)
        output_layer = self.r_fc3(level_2_layer)
        return output_layer
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        pred_y = self.regressor(mu)
        return x_hat, mu, logvar, z, pred_y

### Define loss functions

In [9]:
def vae_loss_function(x_hat, x, mean, log_var): # recon loss and kld loss
        bce = torch.nn.functional.binary_cross_entropy(x_hat, x, reduction = 'sum')
        kld = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
        loss = kld + bce
        return loss, kld, bce

In [10]:
def rmse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = torch.sqrt(loss_fn(y_pred, y))
    return loss

In [11]:
def mse_loss_function(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    loss = loss_fn(y_pred, y)
    return loss

In [12]:
def mse_loss_function_mean(y_pred, y):
    loss_fn = torch.nn.MSELoss(reduction='mean')
    loss = loss_fn(y_pred, y)
    return loss

### Validation functions

In [13]:
def r_squared(y_true, y_pred):
    y_true_mean = torch.mean(y_true)
    SS_res = torch.sum(torch.square(y_true - y_pred))
    SS_tot = torch.sum(torch.square(y_true - y_true_mean))
    r_squared = 1 - SS_res / (SS_tot + torch.finfo(torch.float32).eps)
    return r_squared

### Training parameters

In [14]:
epochs = 50

lr = 0.001
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

k = 10
splits = KFold(n_splits=k, shuffle=False)

In [15]:
# Initialize the model
input_size = 57820 #dimension of gene expressions
level_2 = 2048
level_3 = 1024
latent_dim = 128 # target latent size
model = VAE_with_regressor(input_size, level_2, level_3, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

### Train Function

In [16]:
def train (model, dataloader, epoch, optimizer, w_recon_loss, w_kl_loss, w_reg_loss):
    model.train()
    train_recon = 0
    train_kl  = 0
    train_reg = 0
    train_total_loss = 0
    batch_num = 0

    for batch_idx, batch in enumerate(dataloader):
        batch_num = batch_num + 1
        # get data values
        gene_exp_values, drug_resp_labels = batch
        gene_exp_values = gene_exp_values.to(device)
        drug_resp_labels = drug_resp_labels.to(device)

        # clear gradients
        optimizer.zero_grad()
        # apply model
        x_hat, mu, logvar, z, pred_y = model(gene_exp_values)

        # losses
        vae_loss, kld, bce = vae_loss_function(x_hat, gene_exp_values, mu, logvar)
        mse_loss = mse_loss_function_mean(pred_y, drug_resp_labels)

        #apply loss weights
        loss = w_recon_loss * bce + w_kl_loss * kld + w_reg_loss * mse_loss

        

        # backward
        loss.backward()

        with torch.no_grad():
            train_recon += bce.item()
            train_kl += kld.item()
            train_reg += mse_loss.item()
            train_total_loss += loss.item()

        # update parameters
        optimizer.step()

    train_recon_ave = train_recon/ len(dataloader.sampler)
    train_kl_ave = train_kl/ len(dataloader.sampler)
    train_reg_ave = train_reg/ batch_num
    train_total_ave = train_total_loss/ len(dataloader.sampler)

    print('=====> Epoch {} \n' 
          'Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average MSE Loss: {:.3f} \n'.format(epoch, train_recon_ave, train_kl_ave, train_reg_ave))
    
    return train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave


### Validation function

In [17]:
def test(dataloader, model):
    model.eval()
    test_recon = 0
    test_kl  = 0
    test_reg = 0
    test_rmse = 0
    test_r2 = 0
    batch_num = 0
    with torch.no_grad():
        mean_store = torch.zeros(1, latent_dim).to(device)
        pred_store = torch.zeros(1,10).to(device)
        for batch_idx, batch in enumerate(dataloader):
            batch_num = batch_num + 1
            # get data values
            gene_exp_values, drug_resp_labels = batch
            gene_exp_values = gene_exp_values.to(device)
            drug_resp_labels = drug_resp_labels.to(device)

            # apply model
            x_hat, mu, logvar, z, pred_y = model(gene_exp_values)

            # loss functions
            vae_loss, kld, bce = vae_loss_function(x_hat, gene_exp_values, mu, logvar)
            # bce = mse_loss_function(x_hat, gene_exp_values)
            mse_loss = mse_loss_function(pred_y, drug_resp_labels)
            r2_value = r_squared(drug_resp_labels, pred_y)

            test_recon += bce.item()
            test_kl += kld.item()
            test_reg += mse_loss.item()
            test_r2 += r2_value.item()

            # Get relevant values
            mean_store = torch.cat((mean_store, mu), 0)
            pred_store = torch.cat((pred_store, pred_y), 0)

        # ====== export reduced dimensions and predictions ======
        mean_final = mean_store[1:]
        mean_final_np = mean_final.cpu().numpy()
        mean_final_df = pd.DataFrame(mean_final_np)

        pred_final = pred_store[1:]
        pred_final_np = pred_final.cpu().numpy()
        pred_final_df = pd.DataFrame(pred_final_np)

        input_path = 'C:/Users/mdzak/Desktop/GitHub/FYP_Zaki/notebooks_new/output/'

        mean_path =  input_path + str(latent_dim) + 'D_gene_exp_latent_code.tsv'
        pred_path = input_path + str(latent_dim) + 'D_gene_exp_predictions.tsv'

        mean_final_df.to_csv(mean_path, sep='\t')
        pred_final_df.to_csv(pred_path, sep='\t')

        # print loss
        test_recon_ave = test_recon/ len(dataloader.sampler)
        test_kl_ave = test_kl/ len(dataloader.sampler)
        test_reg_ave = test_reg/ (len(dataloader.sampler) * 10)
        test_rmse_ave = math.sqrt(test_reg_ave)
        test_r2_ave = test_r2/ batch_num

        print('Average Recon Loss: {:.3f} \n'
          'Average KL Loss: {:.3f} \n'
          'Average MSE Loss: {:.3f} \n'
          'Average RMSE Loss: {:.3f} \n'
          'Average R2: {:.3f} \n'.format(test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave))
        
        return test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave

### Execute Training (Cross Validation)

In [18]:
history_train = {'Train Recon Loss': [], 'Train KL Loss': [], 'Train MSE Loss': [], 'Train Total Loss': []}
history_test = {'Test Recon Loss': [], 'Test KL Loss': [], 'Test MSE Loss': [], 'Test RMSE Loss': [], 'Test R2': []}

for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(full_dataset)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(full_dataset, batch_size=batch_size, sampler=test_sampler)

    print("Start Training (Unsupervised Phase)")
    for epoch in range(1, epochs + 1):
       train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave = train(model,train_loader, epoch, optimizer, w_recon_loss=1, w_kl_loss=1, w_reg_loss=0)
       history_train['Train Recon Loss'].append(train_recon_ave)
       history_train['Train KL Loss'].append(train_kl_ave)
       history_train['Train MSE Loss'].append(train_reg_ave)
       history_train['Train Total Loss'].append(train_total_ave)

    print("Start Training (Supervised Phase)")
    for epoch in range(1, epochs + 1):
        train_recon_ave, train_kl_ave, train_reg_ave, train_total_ave = train(model,train_loader, epoch, optimizer, w_recon_loss=0, w_kl_loss=0, w_reg_loss=1)
        history_train['Train Recon Loss'].append(train_recon_ave)
        history_train['Train KL Loss'].append(train_kl_ave)
        history_train['Train MSE Loss'].append(train_reg_ave)
        history_train['Train Total Loss'].append(train_total_ave)

    test_recon_ave, test_kl_ave, test_reg_ave, test_rmse_ave, test_r2_ave = test(test_loader, model)
    history_test['Test Recon Loss'].append(test_recon_ave)
    history_test['Test KL Loss'].append(test_kl_ave)
    history_test['Test MSE Loss'].append(test_reg_ave)
    history_test['Test RMSE Loss'].append(test_rmse_ave)
    history_test['Test R2'].append(test_r2_ave)

Fold 1
Start Training (Unsupervised Phase)
=====> Epoch 1 
Average Recon Loss: 44894.709 
Average KL Loss: 129.261 
Average MSE Loss: 13.289 

=====> Epoch 2 
Average Recon Loss: 43854.121 
Average KL Loss: 126.094 
Average MSE Loss: 13.327 

=====> Epoch 3 
Average Recon Loss: 43744.039 
Average KL Loss: 119.753 
Average MSE Loss: 13.499 

=====> Epoch 4 
Average Recon Loss: 42942.195 
Average KL Loss: 118.426 
Average MSE Loss: 13.499 

=====> Epoch 5 
Average Recon Loss: 42160.231 
Average KL Loss: 115.901 
Average MSE Loss: 13.417 

=====> Epoch 6 
Average Recon Loss: 41734.939 
Average KL Loss: 107.730 
Average MSE Loss: 13.368 

=====> Epoch 7 
Average Recon Loss: 41121.612 
Average KL Loss: 112.500 
Average MSE Loss: 13.592 

=====> Epoch 8 
Average Recon Loss: 40619.996 
Average KL Loss: 111.093 
Average MSE Loss: 13.526 

=====> Epoch 9 
Average Recon Loss: 39936.623 
Average KL Loss: 114.493 
Average MSE Loss: 13.392 

=====> Epoch 10 
Average Recon Loss: 39559.702 
Average K

### Performance of CV

In [19]:
train_avg_cv_recon_loss = np.mean(history_train['Train Recon Loss'])
train_avg_cv_kl_loss = np.mean(history_train['Train KL Loss'])
train_avg_cv_mse_loss = np.mean(history_train['Train MSE Loss'])
train_avg_cv_total_loss = np.mean(history_train['Train Total Loss'])

test_avg_recon_loss =  np.mean(history_test['Test Recon Loss'])
test_avg_kl_loss =  np.mean(history_test['Test KL Loss'])
test_avg_mse_loss =  np.mean(history_test['Test MSE Loss'])
test_avg_rmse_loss =  np.mean(history_test['Test RMSE Loss'])
test_avg_r2_loss =  np.mean(history_test['Test R2'])

print('Performance of {} fold cross validation {} dimension:'.format(k, latent_dim))
print("\nAverage Training Recon Loss: {:.4f} \n Average Training KL Loss: {:.4f} \n Average Training RMSE Loss: {:.3f} \n Average Training Total Loss: {:.3f} \n".format(train_avg_cv_recon_loss, train_avg_cv_kl_loss, train_avg_cv_mse_loss, train_avg_cv_total_loss))
print("\nAverage Testing Recon Loss: {:.4f} \n Average Testing KL Loss: {:.4f} \n Average Testing MSE Loss: {:.3f} \n Average Testing RMSE Loss: {:.3f} \n Average Testing R2 Loss: {:.3f} \n".format(test_avg_recon_loss, test_avg_kl_loss, test_avg_mse_loss, test_avg_rmse_loss, test_avg_r2_loss))  


Performance of 10 fold cross validation 128 dimension:

Average Training Recon Loss: 5830.1211 
 Average Training KL Loss: 15.3085 
 Average Training RMSE Loss: 4.785 
 Average Training Total Loss: 3403.060 


Average Testing Recon Loss: 4970.9288 
 Average Testing KL Loss: 9.2613 
 Average Testing MSE Loss: 3.929 
 Average Testing RMSE Loss: 1.922 
 Average Testing R2 Loss: 0.554 



In [20]:
np.mean(history_test['Test RMSE Loss'])

1.921988737576514

In [21]:
history_test['Test RMSE Loss']

[2.9715656504228085,
 2.4375194856447258,
 2.050654068026016,
 2.0567048763199733,
 2.0824547300981084,
 1.5857064649382775,
 1.7876916600337338,
 1.4446259083021422,
 1.4995654571674626,
 1.3033990748118882]

In [22]:
history_test['Test MSE Loss']

[8.830202414772728,
 5.941501242897727,
 4.205182106711648,
 4.230034948278356,
 4.336617702907986,
 2.5144649929470484,
 3.1958414713541665,
 2.086944014937789,
 2.248696560329861,
 1.6988491482204862]

### Save model

In [23]:
# torch.save(model.state_dict(), 'trained_models/VAE_gene_exp' + str(latent_dim) + '.pt')